In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
def plot_matrix(vector, title=None, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    sns.heatmap(vector, xticklabels=False, yticklabels=False, annot=True, fmt=".2f", cmap='YlGnBu', cbar=False, linewidths=2, ax=ax)
    ax.set_title(title)

def generate_random_vector(N_processors: float, M_tasks: int, T_min: int, T_max: int) -> np.array:
    task_vector = np.random.randint(T_min, T_max, (1, M_tasks))
    return task_vector

def create_manual_matrix(M_tasks: float, N_processors: int) -> np.array:
    matrix = np.zeros((M_tasks, N_processors))
    for i in range(M_tasks):
        matrix = np.append(matrix, [int(i) for i in input(f"Введите через пробел {N_processors} значений строки №{i}: \n").split(' ')])
    return matrix

def create_matrix_from_file(path_to_file):
    file = open(path_to_file)
    for line in file.readlines():
        print(line)
        

def get_user_input(N_processors=None, M_tasks=None, T_min=None, T_max=None, auto=True, path_to_file=None):
        N_processors, M_tasks = [int(i) for i in input("Введите число устройств и количество работ через пробел: ").split(" ")]
        if not auto:
            return create_manual_matrix(M_tasks, N_processors)
        else:
            T_min, T_max = [int(i) for i in input ("Введите через пробел минимальное и максимальное значения для автогенерации: ").split(" ")]
            matrix = np.zeros((M_tasks, N_processors))
            #print(matrix, matrix.shape)
            for i in range(0, M_tasks):
                matrix[i] = generate_random_vector(M_tasks, N_processors, T_min, T_max)
            return matrix

def sort_matrix(matrix):
    row_sums = matrix.sum(axis=1)
    sorted_indices = np.argsort(row_sums)[::-1]
    return matrix[sorted_indices]

In [212]:
matrix = get_user_input(auto=True)
#matrix = np.genfromtxt('matrix.txt', delimiter=',', skip_header=0, deletechars='[]')
sorted_matrix_desc = sort_matrix(matrix)
sorted_matrix_asc = sorted_matrix_desc[::-1]
matrix

array([[12., 13., 12., 18.],
       [13., 14., 14., 15.],
       [14., 14., 14., 13.],
       [14., 11., 18., 11.],
       [11., 11., 17., 19.],
       [12., 14., 10., 18.],
       [15., 11., 19., 15.],
       [14., 13., 11., 12.],
       [11., 15., 14., 16.],
       [16., 12., 19., 16.],
       [14., 16., 11., 18.]])

In [ ]:
sorted_matrix_desc

In [149]:
"""
matrix = np.array([17, 15, 15, 19,
                   15, 19, 22, 21,
                   15, 21, 19, 21,
                   20, 23, 21, 15,
                   17, 17, 17, 20,
                   14, 10, 14, 23,
                   18, 15, 13, 15,
                   20, 17, 12, 22,
                   21, 18, 21, 17,
                   17, 13, 20, 11,
                   23, 17, 14, 17]).reshape(-1, 4)
pd.DataFrame(matrix)
"""

,0,1,2,3
0,17,15,15,19
1,15,19,22,21
2,15,21,19,21
3,20,23,21,15
4,17,17,17,20
5,14,10,14,23
6,18,15,13,15
7,20,17,12,22
8,21,18,21,17
9,17,13,20,11


In [ ]:
'''
fig, ax = plt.subplots(ncols=3, figsize=(10, 15))
plot_matrix(matrix, title='Исходная' , ax=ax[0])
plot_matrix(sorted_matrix_desc, title='В порядке убывания сумм', ax=ax[1])
plot_matrix(sorted_matrix_asc, title='В порядке возрастания сумм', ax=ax[2])
'''

In [150]:
def create_shedule_minim_method(matrix):
    task_matrix = np.zeros(matrix.shape)
    for i in range(matrix.shape[0]):
        min_elem_pos = np.argmin(matrix[i])
        task_matrix[i][min_elem_pos] = matrix[i][min_elem_pos]
    print(task_matrix)
    task_matrix = np.append(task_matrix, task_matrix.sum(axis=0, keepdims=True), axis=0)
    return task_matrix

def create_shedule(matrix, mult):
    task_matrix = np.zeros(matrix.shape)
    support_matrix = np.zeros(matrix.shape)
    matrix = np.copy(matrix)
    for i in range(matrix.shape[0]):
        cols_sum = task_matrix.sum(axis=0)
        #print("Сумма по столбцам", cols_sum)
        sq_sum = np.power(cols_sum + matrix[i], mult)
        #print("Выбор из: ", sq_sum)
        support_matrix[i] = sq_sum
        min_elem_pos = np.argmin(sq_sum)
        #print(cols_sum[min_elem_pos], matrix[i][min_elem_pos], task_matrix)
        #print("Предыдущий в этом: ", sq_sum[min_elem_pos])
        #print("Минимальный элемент: ", matrix[i][min_elem_pos])
        print(cols_sum[min_elem_pos])
        task_matrix[i][min_elem_pos] = (matrix[i][min_elem_pos])
        #print(f"Задания на строке: {task_matrix[i]} Сумма предыдущих {cols_sum}")

    task_matrix = np.append(task_matrix, task_matrix.sum(axis=0, keepdims=True), axis=0)
    support_matrix = np.append(support_matrix, support_matrix.sum(axis=0, keepdims=True), 0)
    #print(task_matrix)
    #print(support_matrix)
    result = pd.concat([pd.DataFrame(task_matrix), pd.DataFrame(support_matrix)], axis=1)
    return task_matrix, support_matrix, result
 

In [192]:
def create_power_shedule(matrix, mult):
    task_matrix = np.zeros(matrix.shape)
    support_matrix = np.zeros(matrix.shape)
    matrix = np.copy(matrix)
    for i in range(matrix.shape[0]):
        cols_sum = task_matrix.sum(axis=0)
        #print("Сумма по столбцам", cols_sum)
        sq_sum = np.array(matrix[i])
        for j in range(len(cols_sum)):
            elem = np.zeros(len(cols_sum), dtype=np.float64)
            elem[j] = matrix[i][j]
            sq_sum[j] = np.sum(np.power(elem + cols_sum, mult))
            print(elem + cols_sum)

        #sq_sum = np.power(cols_sum + matrix[i], mult)
        print("Выбор из: ", sq_sum)
        support_matrix[i] = sq_sum
        min_elem_pos = np.argmin(sq_sum)
        #print(cols_sum[min_elem_pos], matrix[i][min_elem_pos], task_matrix)
        #print("Предыдущий в этом: ", sq_sum[min_elem_pos])
        #print("Минимальный элемент: ", matrix[i][min_elem_pos])
        print(cols_sum[min_elem_pos])
        task_matrix[i][min_elem_pos] = (matrix[i][min_elem_pos])
        #print(f"Задания на строке: {task_matrix[i]} Сумма предыдущих {cols_sum}")

    task_matrix = np.append(task_matrix, task_matrix.sum(axis=0, keepdims=True), axis=0)
    support_matrix = np.append(support_matrix, support_matrix.sum(axis=0, keepdims=True), 0)
    #print(task_matrix)
    #print(support_matrix)
    result = pd.concat([pd.DataFrame(task_matrix), pd.DataFrame(support_matrix)], axis=1)
    return task_matrix, support_matrix, result
 

In [213]:
shedule = create_power_shedule(matrix, 2)
#shedule_for_desc = create_shedule(sorted_matrix_desc)
#shedule_for_asc = create_shedule(sorted_matrix_asc)
'''
print(f"Результат матрицы случайной: max({shedule[-1]}) = {max(shedule[-1])}")
print(f"Результат матрицы по убыванию: max({shedule_for_desc[-1]}) = {max(shedule_for_desc[-1])}")
print(f"Результат матрицы по возастанию: max({shedule_for_asc[-1]}) = {max(shedule_for_asc[-1])}")
''';


[12.  0.  0.  0.]
[ 0. 13.  0.  0.]
[ 0.  0. 12.  0.]
[ 0.  0.  0. 18.]
Выбор из:  [144. 169. 144. 324.]
0.0
[25.  0.  0.  0.]
[12. 14.  0.  0.]
[12.  0. 14.  0.]
[12.  0.  0. 15.]
Выбор из:  [625. 340. 340. 369.]
0.0
[26. 14.  0.  0.]
[12. 28.  0.  0.]
[12. 14. 14.  0.]
[12. 14.  0. 13.]
Выбор из:  [872. 928. 536. 509.]
0.0
[26. 14.  0. 13.]
[12. 25.  0. 13.]
[12. 14. 18. 13.]
[12. 14.  0. 24.]
Выбор из:  [1041.  938.  833.  916.]
0.0
[23. 14. 18. 13.]
[12. 25. 18. 13.]
[12. 14. 35. 13.]
[12. 14. 18. 32.]
Выбор из:  [1218. 1262. 1734. 1688.]
12.0
[35. 14. 18. 13.]
[23. 28. 18. 13.]
[23. 14. 28. 13.]
[23. 14. 18. 31.]
Выбор из:  [1914. 1806. 1678. 2010.]
18.0
[38. 14. 28. 13.]
[23. 25. 28. 13.]
[23. 14. 47. 13.]
[23. 14. 28. 28.]
Выбор из:  [2593. 2107. 3103. 2293.]
14.0
[37. 25. 28. 13.]
[23. 38. 28. 13.]
[23. 25. 39. 13.]
[23. 25. 28. 25.]
Выбор из:  [2947. 2926. 2844. 2563.]
13.0
[34. 25. 28. 25.]
[23. 40. 28. 25.]
[23. 25. 42. 25.]
[23. 25. 28. 41.]
Выбор из:  [3190. 3538. 3543. 36

In [214]:
print("Исходная матрица:", pd.DataFrame(matrix), sep='\n')
print(f"Результат матрицы случайной: max({shedule[0][-1]}) = {max(shedule[0][-1])}")
shedule[2]

Исходная матрица:
       0     1     2     3
0   12.0  13.0  12.0  18.0
1   13.0  14.0  14.0  15.0
2   14.0  14.0  14.0  13.0
3   14.0  11.0  18.0  11.0
4   11.0  11.0  17.0  19.0
5   12.0  14.0  10.0  18.0
6   15.0  11.0  19.0  15.0
7   14.0  13.0  11.0  12.0
8   11.0  15.0  14.0  16.0
9   16.0  12.0  19.0  16.0
10  14.0  16.0  11.0  18.0
Результат матрицы случайной: max([34. 37. 39. 25.]) = 39.0


,0,1,2,3,0,1,2,3
0,12.0,0.0,0.0,0.0,144.0,169.0,144.0,324.0
1,0.0,14.0,0.0,0.0,625.0,340.0,340.0,369.0
2,0.0,0.0,0.0,13.0,872.0,928.0,536.0,509.0
3,0.0,0.0,18.0,0.0,1041.0,938.0,833.0,916.0
4,11.0,0.0,0.0,0.0,1218.0,1262.0,1734.0,1688.0
5,0.0,0.0,10.0,0.0,1914.0,1806.0,1678.0,2010.0
6,0.0,11.0,0.0,0.0,2593.0,2107.0,3103.0,2293.0
7,0.0,0.0,0.0,12.0,2947.0,2926.0,2844.0,2563.0
8,11.0,0.0,0.0,0.0,3190.0,3538.0,3543.0,3619.0
9,0.0,12.0,0.0,0.0,4534.0,3934.0,4615.0,4246.0


In [196]:

shedule_ord = create_shedule(matrix, 1)
print(f"Результат матрицы случайной: max({shedule_ord[0][-1]}) = {max(shedule_ord[0][-1])}")
shedule_ord[2]

0.0
0.0
0.0
0.0
15.0
15.0
15.0
19.0
25.0
30.0
31.0
Результат матрицы случайной: max([32. 43. 45. 41.]) = 45.0


,0,1,2,3,0,1,2,3
0,0.0,15.0,0.0,0.0,17.0,15.0,15.0,19.0
1,15.0,0.0,0.0,0.0,15.0,34.0,22.0,21.0
2,0.0,0.0,19.0,0.0,30.0,36.0,19.0,21.0
3,0.0,0.0,0.0,15.0,35.0,38.0,40.0,15.0
4,17.0,0.0,0.0,0.0,32.0,32.0,36.0,35.0
5,0.0,10.0,0.0,0.0,46.0,25.0,33.0,38.0
6,0.0,0.0,0.0,15.0,50.0,40.0,32.0,30.0
7,0.0,0.0,12.0,0.0,52.0,42.0,31.0,52.0
8,0.0,18.0,0.0,0.0,53.0,43.0,52.0,47.0
9,0.0,0.0,0.0,11.0,49.0,56.0,51.0,41.0
